# IBM Data Science Professional Certificate capstone project
This notebook contains work towards the Professional Certificate capstone project

In [2]:
import pandas as pd
import numpy as np

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Segmenting and Clustering Neighborhoods in Toronto

## Import the necessary dependencies

In [110]:
from bs4 import BeautifulSoup
import requests

## Fetch the website with the information

In [111]:
# Define url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [112]:
# Get text trhough a request
page = requests.get(url).text

In [113]:
# Create a BeautifulSoup object
soup = BeautifulSoup(page, "html.parser")

In [115]:
# Check whether we got the right page
print(soup.title)

<title>List of postal codes of Canada: M - Wikipedia</title>


## Extract useful information from the table rows

In [117]:
# Create an empty list which will be populated with postal code - borough - neighborhood triplets
contents = []

# Fetch only the table from the webpage
table = soup.find('table')

# Iterate over rows in the table and collect relevant information
for row in table.findAll('td'):
    # Create an empty dictionary per iteration
    dict1 = {}
    if row.span.text == 'Not assigned':
        pass
    # Populate dictionaries with data
    else:
        dict1['postal_code'] = row.p.text[:3]
        dict1['borough'] = (row.span.text).split('(')[0]
        dict1['neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
    # Append dictionary data (postal code - borough - neighborhood triplet) to the empty list
        contents.append(dict1)

## Create a dataframe

In [118]:
# Create a dataframe from the contents list
df_toronto = pd.DataFrame(contents)

In [119]:
# Visualize both ends of the dataframe
df_toronto.head()

,postal_code,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [120]:
df_toronto.tail()

,postal_code,borough,neighborhood
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [121]:
# Correct some borough names, then visualize again
df_toronto['borough'] = df_toronto['borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest',
                                                       'East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [129]:
df_toronto.head()

,postal_code,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [130]:
df_toronto.tail()

,postal_code,borough,neighborhood
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [147]:
# Check whether any postal code entry appears more than once
false = []
true = []
for item in df_toronto['postal_code'].value_counts() > 1:
    if item == False:
        false.append(False)
    if item == True:
        true.append(True)
print('Repeated postal codes: ' + str(len(true)))
print('Unique postal codes: ' + str(len(false)))

Repeated postal codes: 0
Unique postal codes: 103


In [164]:
# Check whether any borough has no assigned neighborhoods
assigned = []
not_assigned = []
for item in df_toronto['neighborhood'].str.contains('Not assigned'):
    if item == False:
        assigned.append(False)
    if item == True:
        not_assigned.append(True)

print('boroughs with no assigned neighborhoods: ' + str(len(not_assigned)))
print('boroughs with assigned neighborhoods: ' + str(len(assigned)))

boroughs with no assigned neighborhoods: 0
boroughs with assigned neighborhoods: 103
